# Welcome to AJ's Blue Carbon Processing tools!

First, select the line (with imports, authenticate, and initialize). Wait ~10 seconds, and a tab will open where you must follow a series of steps to get a code. Copy that code, enter it in the box that says "Enter verification code:", and hit enter. You will then be able to use the tool!

In [1]:
import geemap
import ee
import Main_script
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1ASc3gC3znRXtMZh8DjnEJcuph2Hg1h4O4FeGW9Ac77k2-_ZPPXV5PJx-7XA

Successfully saved authorization token.


Now, you should see the words "Successfully saved authorization token." above. This should now work!

# Basic Tools

This section includes the most basic of uses for your project. Hopefully, this will be all you have to use. There are 2 functions below: 
1) get_csv

This exports a CSV file with all of your basic, key information in a table, based on your project shapefile, project starting year, sedimentation as cm/year, and output folder. Please input all your information in the box below, and with that box selected, hit run at the top of the screen.

PLEASE NOTE that if your project starts in november or december of a given year, you should input the year after that to get your proper baseline (i.e. for project start in November 2015, start_year should be 2016).

2) get_tifs

This exports several .tif files of various information (gmw, JAXA, DEM, and submergence) into a given folder output of your choice.

In [42]:
#Please input the path of your shapefile, surrounded with quotation marks. 
#This must be in an UNZIPPED folder of your shapefile, and end in .shp or .kml. 
#Please note .kml files have not yet been tested with this software.
filepath = 'VCS2250/Calyx_VCS_2250_project.shp'
#Please put your project's starting year below. As stated above, if project started in Nov or Dec of year, input the following year.
starting_year = 2018
#Input sedimentation below in cm/year
sedimentation = 0.06
#Finally, please input the name of your desired output folder below.
output_folder = "outputs"

#Once all of these are input, hit run on this cell.

In [54]:
#If you want to output your informational CSV file, run this cell. This should take about a minute.
#Once the * to the left of this box turns into a number, your output csv should be in your chosen folder!
Main_script.get_csv(filepath, starting_year, sedimentation, output_folder)

NameError: name 'get_area' is not defined

In [6]:
#If you want to output all necessary .tif files for maps, run this cell.
main.get_tifs(filepath, starting_year, sedimentation, output_folder)

Percentage of total: 34.72055785257141%.


FUNCTIONS - to be transferred to main.py

In [5]:
def convert_to_ee(filepath):
    """
    Loads a vector file into earth engine as a feature collection/image.
    Supports .shp and .kml. .kml files are untested.    
    """
    
    filepath = filepath.lower()
    
    if filepath.endswith('.shp'):
        return geemap.shp_to_ee(filepath)
    elif filepath.endswith('.kml'):
        return geemap.kml_to_ee(filepath)
    else:
        raise ValueError("Unsupported file type - must be .shp or .kml")

In [43]:
def get_csv(filepath, start_year, sedimentation, folder):
    #Parse input data
    aoi = convert_to_ee(filepath)
    eval_year = start_year - 1
    year_string = str(eval_year)
    
    #Call functions for csv data
    area = round(get_area(aoi), 2)
    
    murray_hectares_dict = Baseline.murray_hectares(aoi, eval_year)
    murray_percent_dict = Baseline.murray_percent(aoi, eval_year)
    
    gmw_hectares = Baseline.gmw_hectares(aoi, eval_year)
    gmw_percent = Baseline.gmw_percent(aoi, eval_year)

    jaxa_hectares = Baseline.jaxa_hectares(aoi, eval_year)
    jaxa_percentages = Baseline.jaxa_percent(aoi, eval_year)
    
    slr_dict = SLR.get_slr_dictionary(aoi, eval_year)
    
    elevation_dict = SLR.get_elevation_data(aoi)
    
    #Get SLR value from 100 years in future
    ssp370_last_year_SLR = slr_dict["SSP3-7.0"][(((start_year // 10) + 1) * 10)+100]
    ssp585_last_year_SLR = slr_dict["SSP5-8.5"][(((start_year // 10) + 1) * 10)+100]
    
    #No data parsing needed for sedimentation: 1 cm/year = 1 meter/100 years! It took me way too long to realize that.
    inundation_height_ssp370 = SLR.calculate_inundation_height(sedimentation, ssp370_last_year_SLR) 
    area_inundated_hectares_ssp370 = SLR.area_inundated_hectares(aoi, inundation_height_ssp370)
    area_inundated_percent_ssp370 = SLR.area_inundated_percent(aoi, inundation_height_ssp370)
    
    inundation_height_ssp585 = SLR.calculate_inundation_height(sedimentation, ssp585_last_year_SLR)
    area_inundated_hectares_ssp585 = SLR.area_inundated_hectares(aoi, inundation_height_ssp585)
    area_inundated_percent_ssp585 = SLR.area_inundated_percent(aoi, inundation_height_ssp585)
    
    #Finally, protected planet data.
    protected_planet_hectares = PP.protected_planet_hectares(aoi)
    protected_planet_percent = PP.protected_planet_percent(aoi)
    
    #CREATE ROWS FROM DATA COLLECTED ABOVE - MOSTLY VISUALS AND AESTHETICS OF SHEET BELOW.
    
    #BASELINE ROWS - complicated to make it look pretty - don't worry about below unless you want to change the CSV format.
    #includes Murray and GMW
    baseline_rows = [
        ["Baseline Spatial Analysis"],
        ['Project Area:', area, 'ha'],
        ['Murray tree cover loss (' + str(start_year - 10) + '-' + year_string + ')', round(murray_hectares_dict['ten_year_loss'], 2),
                'ha', str(murray_percent_dict['ten_year_loss_percent']) + "%", "Note that data cuts off at 2019."],
        ['Murray tree cover loss (1999-2019)', round(murray_hectares_dict['total'], 2),
                'ha', str(murray_percent_dict['total_loss_percent']) + "%"],
        ['Area covered by forest ' + year_string + " GMW", round(gmw_hectares, 2), 'ha', str(gmw_percent) + "%"],
    ]
    
    #initalize jaxa_rows
    jaxa_rows = None
    
    #Make jaxa rows - some parsing must be done
    if type(jaxa_hectares) is dict:
        jaxa_rows = [
            ['Area covered by dense forest ' + year_string + ' JAXA', round(jaxa_hectares["Dense"], 2), "ha", str(jaxa_percentages["Dense"]) + "%"],
            ['Area covered by sparse forest ' + year_string + ' JAXA', round(jaxa_hectares["Non-dense"], 2), "ha", str(jaxa_percentages["Non-dense"]) + "%"],
            ['Area covered by forest total ' + year_string + ' JAXA', round(jaxa_hectares["Total"], 2), "ha", str(jaxa_percentages["Total"]) + "%"],
        ]
    else:
        jaxa_rows = [
            ['Area covered by dense forest ' + year_string + ' JAXA', "N/A"],
            ['Area covered by sparse forest ' + year_string + ' JAXA', "N/A"],
            ['Area covered by forest total ' + year_string + ' JAXA', round (jaxa_hectares["Total"], 2), "ha", str(jaxa_percentages["Total"]) + "%"],
        ]
        
    #Make SLR rows
    
    years = slr_dict["SSP3-7.0"].keys()
    slr_rows = [['SLR DATA','IPCC Sea Level Rise SSP3-7.0 (m)','IPCC SLR SSP5-8.5 (m)']]
    for year in years:
        row = [year, slr_dict["SSP3-7.0"][year], slr_dict["SSP5-8.5"][year]]
        slr_rows.append(row)
        
    #Make Elevation rows
    elevation_rows = [
        ["Elevation of area"],
        ["Mean", round(elevation_dict["mean"],2), "m"],
        ["Min", round(elevation_dict["min"],2), "m"],
        ["Max", round(elevation_dict["max"],2), "m"]
        
    ]
    
    #Make inundation rows
    submergence_rows = [
        ["Project Area lost to SLR after 100 years calculation"],
        ["IPCC Sea Level Rise SSP3-7.0 in Area"],
        [round(area_inundated_hectares_ssp370, 2), "ha under", inundation_height_ssp370, "m"],
        [round(area_inundated_percent_ssp370, 2), "% inundated"],
        ["IPCC Sea Level Rise SSP5-8.5 in Area"],
        [round(area_inundated_hectares_ssp585, 2), "ha under", inundation_height_ssp585, "m"],
        [round(area_inundated_percent_ssp585, 2), "% inundated"]
    ]
    
    #Make Protected Planet Rows
    pp_rows = [
        ["Protected Planet Statistics"],
        [round(protected_planet_hectares, 2), "ha"],
        [round(protected_planet_percent, 2), "ha"]
    ]

    with open(folder + '/output.csv', mode='w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow([])
        writer.writerows(baseline_rows)
        writer.writerows(jaxa_rows)
        writer.writerow([])
        writer.writerows(slr_rows)
        writer.writerow([])
        writer.writerows(elevation_rows)
        writer.writerow([])
        writer.writerows(submergence_rows)
        writer.writerow([])
        writer.writerows(pp_rows)
        
        

In [7]:
def get_tifs(filepath, start_year, sedimentation, folder):

SyntaxError: incomplete input (810434216.py, line 1)

In [52]:
import Main_script

In [ ]:
#MUST DO - make sure jaxa year is valid - if not, do most recent year BEFORE then.